# Running MD with LAMMPS

In this notebook, we will use LAMMPS (https://lammps.org/) to run a Molecular
Dynamics simulation, using the exact same potential we just exported and used
from ASE.

This demonstrate how metatensor models can be used from multiple MD engines in
the same way. Additionally, this shows that while the models where defined using
Python, they can be loaded and used from a pure C++ simulation engine through
TorchScript.

In [ ]:
import os
import subprocess

import chemiscope
import ase.io


If you are **NOT** running on CSC Puhti, please change the path below to the path of
your LAMMPS installation including `pair_style metatensor`.

In [ ]:
LAMMPS_EXE = "/projappl/project_2008666/metatensor/env/bin/lmp"


To run MD with LAMMPS, we'll need two input files: one defining the initial
conformation of the atoms and another one defining the simulation settings.

In [ ]:
# Write the input structure using ase
frame = ase.io.read("ethanol_conformers_dftb.xyz", "0")
frame.cell = [12, 12, 12]
ase.io.write(
    os.path.join("simulation", "ethanol.lmp"),
    frame,
    format="lammps-data",
    atom_style="full",
    specorder=["H", "C", "O"],
)

with open(os.path.join("simulation", "ethanol.lmp")) as fd:
    print(fd.read())


In [ ]:
# Write the LAMMPS input manually

LAMMPS_INPUT = """
# use Angstroms, eV, ps as units
units metal
# use non-periodic, fixed boundaries
boundary f f f

# read the data we just wrote with ASE
atom_style full
read_data ethanol.lmp

# set atoms masses
mass 1 1.008
mass 2 12.011
mass 3 15.999

# Potential definition, using a custom pair style.
# This will load the model we just exported
pair_style metatensor ../ethanol-model-with-lj.pt

# map from LAMMPS atoms types to the species in the model
#          LAMMPS LAMMPS        SPECIES
pair_coeff    1     1              1
pair_coeff    2     2              6
pair_coeff    3     3              8

timestep 0.001  # 1 fs timestep
fix 1 all nve   # use NVE ensemble integration


# output information every step: temperature, potential energy, total energy
thermo 1
thermo_style custom step temp pe etotal


# Output the trajectory in XYZ format, using actual atom names
# instead of LAMMPS numeric types
dump 1 all xyz 1 trajectory.xyz
dump_modify 1 element H C O

# Run the simulation for 80 steps
run 80
"""

with open(os.path.join("simulation", "run.in"), "w") as fd:
    fd.write(LAMMPS_INPUT)


We can now start the simulation in a sub-process, running LAMMPS with the input
file we defined above.

In [ ]:
subprocess.run(
    [LAMMPS_EXE, "-in", "run.in"],
    cwd="simulation"
)


Let's visualize the trajectory!

In [ ]:
trajectory = ase.io.read(os.path.join("simulation", "trajectory.xyz"), ":", format="xyz")

chemiscope.show(
    trajectory, mode="structure", settings={"structure": [{"playbackDelay": 50}]}
)
